<a href="https://colab.research.google.com/github/1900690/gradio/blob/main/gradio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Google Colab上でwebアプリが作れるGradioの使い方](https://clameyes.com/posts/google-colab-web-app-gradio)

##※認証がある場合はID;test pass:gradio

In [1]:
#@title gradioライブラリをインストール
#@markdown この共有リンクの有効期限は1週間です。永続的なホスティングとGPUのアップグレードを無料で行うには、作業ディレクトリのターミナルから `gradio deploy` を実行し、Hugging Face Spaces (https://huggingface.co/spaces) にデプロイしてください。
!pip -q install gradio

In [ ]:
#@title 指定したテキストを返すアプリ
import gradio as gr

# 関数を定義
def greet(name):
    return "Hello " + name + "!"

# Webアプリを作成
app = gr.Interface(fn=greet, inputs="text", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2db3a0419addaceb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title スライダーで数を定義するアプリ
import gradio as gr

# 関数を定義
def count(number):
    return "数字は " + str(number) + "!"

# Webアプリを作成
app = gr.Interface(fn=count, inputs="slider", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d38daa5056d6f920f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Google ColabでGUI｜gradioを利用したウェブアプリの作り方を解説](https://tech.aru-zakki.com/gradio-on-google-colab/)

In [2]:
#@title 画像をフィルターに通すアプリ
import numpy as np
import gradio as gr

#セピアフィルターを定義
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

app = gr.Interface(sepia, gr.Image(), "image")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://402e2f7432d4e668b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
#@title　入力した値を覚えるアプリ
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

app = gr.Interface(
    track_score,
    gr.Number(label="Score"),
    gr.JSON(label="Top Scores")
)
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd3b70f4d607c1eddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[garadioのplaygroundにサンプルがたくさん](https://www.gradio.app/playground)